# Lab 4.2 - Chatbot for question-answering

Copyright, Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

In this notebook, we will create a Telegram chatbot that will answer questions, given a predefined question-answer set. 

**Main goal of this notebook**: The most important goal of this notebook is to have a Telegram chatbot that you can ask factual questions to, and receive predefined answers.

**At the end of this notebook, you will**:
* **Use a predefined question - answering dataset** 

## Using a question-answer dictionary

We can use the BotHandler created in the previous notebook. To keep this notebook clear, we moved this to an external file called *utils.py* and import it from there. You can see *utils.py* in the same folder next to this notebook. You can open this file in Jupyter notebook/lab just like a notebook. You can see it has the same functions we have shown before in the introduction notebook.

We can load it in the same way as we import other packages because the notebook will look locally for any Python file with this name. After importing, all the functions defined there can be used in this notebook. Here we specifically import the *read_token* function and *BotHandler* class.

In [1]:
import json
import random

from utils import read_token, BotHandler

For answering question, we are going to load another json file from the data folder that has a structure defined according to the API of Telegram. To inspect this file, you can also double click it and open it in Jupyter notebook/lab.

You see a structure with 17 different items in JSON format. Each item consists of a category, a list of questions and a list of responses. The principle is very simple. If the bot finds a match for the questions, it will generate one of the responses (randomly from the list). Have fun reading through the items. You see it is easy to extend this file with your own reponses to questions.

The following function using the *json* package can load any well-formed JSON file and return its data structure:

In [4]:
def read_qa(qa_path):
    with open(qa_path) as f:
        qa_data = json.load(f)

    return qa_data

We use this function to load the *q-and-a.json* file in addition to the token. We also set the user_id.

In [5]:
CLTL_TOKEN = read_token()
user_id = 408043639
qa_data = read_qa(qa_path = './data/q-and-a.json')

The *json* package provides a way to inspect the data structure that is loaded. Let's look at the first item:

In [6]:
# Explore the structure of the qa_data
print(json.dumps(qa_data['intents'][0], indent=2))

{
  "category": "greeting",
  "questions": [
    "Hi there",
    "How are you",
    "Is anyone there",
    "Hey",
    "Hola",
    "Hello",
    "Good day"
  ],
  "responses": [
    "Hello",
    "Good to see you again",
    "Hi there, how can I help"
  ]
}


We also need a function that can select a response from this structure given any message that is the input.

In [7]:
def create_response(message, qa_data):
    ### We initialise the respond with the cannot answer respond, in case we cannot process the message
    response = "I cannot respond to this"
    
    for i in qa_data['intents']:
        if message in i['questions']:
            response = random.choice(i['responses'])
            break
    
    return response

In [8]:
bot = BotHandler(CLTL_TOKEN)
last_message = bot.get_last_message_by(user_id)
if not last_message:
    last_message = "I am just saying something"
response = create_response(last_message, qa_data)
bot.send_message_to(user_id, response)


print("Received: {message}".format(message=last_message))
print("Responded: {response}".format(response=response))

Received: I am just saying something
Responded: I cannot respond to this


## End of this notebook